In [1]:
import selenium
from selenium import webdriver
from selenium.webdriver import ActionChains

from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait
import time
import requests
from bs4 import BeautifulSoup
import re
import datetime
import pandas as pd
import numpy as np

In [2]:
todaydate = datetime.datetime.today()

year = todaydate.strftime("%y")
month = todaydate.strftime("%m")
day = todaydate.strftime("%d")

In [3]:
url_origin = 'http://www.ssg.com/search.ssg?target=all&query=%EB%96%A1%EB%B3%B6%EC%9D%B4&ctgId=6000092899&ctgLv=1&page={}'

href_list = []


for i in range(1, 6):
    url = url_origin.format(i)
    driver = webdriver.Chrome(executable_path='chromedriver')
    driver.get(url)
    for q in range(1,3):
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(1)

    html_source = driver.page_source
    soup = BeautifulSoup(html_source, 'html.parser')
    ee = soup.find('div', {'class': 'tmpl_itemlist'})
    s = ee.select('div.thmb')

    for q in s:
        x = q.find('a')
        ex = x.get('href')
        href_list.append(ex)
        
    time.sleep(2)    
    driver.close()



In [18]:
ch = []
data = {'name':ch, 'price':ch, 'review':ch,  'grade':ch, 'img_url':ch, 'product_num':ch, 'brand':ch, 'regi':ch, 'link': ch}
df = pd.DataFrame(data)

In [21]:
def datas(soup):
    
    res = []

    name = soup.find('h2', {'class': 'cdtl_info_tit'})
    name_f = name.get_text()

    price = soup.find('em', {'class': 'ssg_price'}).get_text()
    price_f = int(re.sub(r'[^0-9]', '', price))
    
    img1 = soup.find('div', {'class': 'cdtl_item_image'})
    img_url = img1.find('img')
    img_url = img_url.get('src')
    
    review = 0
    grade = 0
    brand_f = np.NaN
    
    try:
        review = soup.find('em', {'id': 'gCommentTotalCnt'}).get_text() 
        review = int(re.sub(r'[^0-9]', '', review))
    except:
        print('리뷰정보 없음')
    
    try:
        grade = float(soup.find('em', {'class': 'cdtl_grade_total'}).get_text())
    except:
        print('별점정보 없음')
        
    check = driver.current_url    
    product_num = check.split('itemId=')[1]
    product_num = product_num.split('&')[0]
    
    
    
    
    try:
        brand = soup.find('span', {'class': 'cdtl_store_tittx'})
        brand_f = brand.get_text()
    except:
        print('브랜드정보 없음')
    
    
    
    res.append(name_f)
    res.append(price_f)
    res.append(review)
    res.append(grade)
    res.append(img_url)
    
    res.append(product_num)
    res.append(brand_f)
    res.append(np.NaN)
    return(res)

In [22]:
for i in range(0, len(href_list)):
    
    print(i)
    url = 'http://www.ssg.com' + href_list[i]
    driver = webdriver.Chrome(executable_path='chromedriver')
    driver.get(url)
    for q in range(1,3):
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(1)
    html_source = driver.page_source
    soup = BeautifulSoup(html_source, 'html.parser')        
    res = datas(soup)
    res.append(driver.current_url)
    
    df.loc[len(df)] = res
    
    driver.close()
    time.sleep(2)

0
리뷰정보 없음
별점정보 없음
1
리뷰정보 없음
별점정보 없음
2
리뷰정보 없음
별점정보 없음
3


Exception ignored in: <function Service.__del__ at 0x0000021312940E50>
Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\lib\site-packages\selenium\webdriver\common\service.py", line 176, in __del__
    self.stop()
  File "C:\ProgramData\Anaconda3\lib\site-packages\selenium\webdriver\common\service.py", line 151, in stop
    self.send_remote_shutdown_command()
  File "C:\ProgramData\Anaconda3\lib\site-packages\selenium\webdriver\common\service.py", line 132, in send_remote_shutdown_command
    if not self.is_connectable():
  File "C:\ProgramData\Anaconda3\lib\site-packages\selenium\webdriver\common\service.py", line 115, in is_connectable
    return utils.is_connectable(self.port)
  File "C:\ProgramData\Anaconda3\lib\site-packages\selenium\webdriver\common\utils.py", line 106, in is_connectable
    socket_ = socket.create_connection((host, port), 1)
  File "C:\ProgramData\Anaconda3\lib\socket.py", line 796, in create_connection
    sock.connect(sa)
KeyboardInterrupt: 

리뷰정보 없음
별점정보 없음


WebDriverException: Message: chrome not reachable
  (Session info: chrome=92.0.4515.159)


In [17]:
def names(x):
    res = False
    if '떡볶이' in x:
        res = True
    return res

df['check'] = df['name'].apply(names)
df1 = df[df['check'] == True]

In [18]:
def names2(x):
    res = x.rsplit('원산지', 1)[0]
    return(res)


df1['name'] = df1['name'].apply(names2)

<ipython-input-18-962576801e3a>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['name'] = df2['name'].apply(names2)


In [19]:
df1['year'] = year
df1['month'] = month
df1['day'] = day
df1['market'] = 'ssg'

<ipython-input-19-f5abdcc7626b>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['year'] = year
<ipython-input-19-f5abdcc7626b>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['month'] = month
<ipython-input-19-f5abdcc7626b>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-v

In [20]:
df1.to_excel('ssg_.xls')

<ipython-input-20-575c293b9594>:1: FutureWarning: As the xlwt package is no longer maintained, the xlwt engine will be removed in a future version of pandas. This is the only engine in pandas that supports writing in the xls format. Install openpyxl and write to an xlsx file instead. You can set the option io.excel.xls.writer to 'xlwt' to silence this warning. While this option is deprecated and will also raise a warning, it can be globally set and the warning suppressed.
  df2.to_excel('ssg_.xls')
